## Raw ideas

so here's a few quick ideas for a validation schema:

- data shouldn't be shuffled before training, cause it's essential to have the time series values in consecutive order
- currently I came up with three ideas for a schema:
    1. `1,2train+3valid` -> `1,2,3train+4valid` -> `1,2,3,4train+5valid` etc.

        **Advatages**:
        - the train set grows, probably helping to test model's robustness
        - we always validate on different parts of the training set

        **Disadvantages**:
        - computationally heavy. at some point our train will contain almost the entire dataset
    2. `1,2train+3valid` -> `2,3train+4valid` -> `3,4train+5valid`

        **Advantages**:
        - fixed train size, no computational power issues
        - we can somewhat efficiently use our dataset since we use each fold several times during different trainig sessions
        
        **Disadvantages**:
        - the fact that different sets overlap might be a problem (shouldn't be, but who knows)
    3.  `1,2train+3valid` -> `4,5train+6valid` -> `7,8train+9valid`

        **Advantages**:
        - the easiest one to implement, goes through each fold once only
        - it's guaranteed that there's no overlapping or data leaks
        
        **Disadvantages**:
        - data hugry in some sense, each fold is used only ones and each trainig session takes $f_{train} + 1$ folds, where $f_{train}$ is the amount of folds for the trainig set. this might be a problem assuming we have only 33 folds at maximum

model ideas:

- **decision tree-based algorithms (boosting, random forest)**:

    seems to be a good choice for the task since we have a lot of categorical features and the target is a discrete variable (which is typical for decision tree regression). at the same time, trees can be overfit easily, which can cause a huge loss in robustness and very inaccurate predictions on unseen data. ensembles will help to resolve this, but the problem of overfitting won't be gone completely. also, decision trees don't work well once something completely unseen shows up in input data, which might be a problem during future usage
- **auto-regressive models**:

    a time-series specific solution, that's supposed to do a good job at predicting future sales based on historical data. the main problems are the following: 
        1. I don't have any experience working with them (but we'll still try I guess)
        2. the data is still quite noisy, so using AR models will require some additional preprocessing 
- **RNNs**:

    one of the hardest models to implement, but at the same time a really powerfull (in theory) solution, that by definition fits the task idea quite well. RNNs can be really good when it comes to analyzing consecitive data (like text processing or time series) 


useful additional features:

- day of the week
- month
- year
- lagged values (look them up in `EDA`)
- ...
